In [8]:
from urllib.parse import unquote
import xml.etree.ElementTree as et

xtree = et.parse("learning_dataset.xml")
xroot = xtree.getroot()
import re
import numpy as np
from numpy import array
import csv
import random
import pandas as pd
import http.client
import email.parser
parser = email.parser.HeaderParser()



docs=[]
labels={'Valid':0, 'XSS':1, 'SqlInjection':2, 'LdapInjection':3,
        'XPathInjection':4, 'PathTransversal':5, 'OsCommanding':6, 'SSI':7}
max_length=0
for node in xroot:
    sample_id = node.attrib.get("id")

    request = node.find('{http://www.example.org/ECMLPKDD}request')
    clazz = node.find('{http://www.example.org/ECMLPKDD}class')
    type_ = clazz.find('{http://www.example.org/ECMLPKDD}type')
    method = request.find('{http://www.example.org/ECMLPKDD}method')
    protocol = request.find('{http://www.example.org/ECMLPKDD}protocol')
    uri = request.find('{http://www.example.org/ECMLPKDD}uri')
    query = request.find('{http://www.example.org/ECMLPKDD}query')
    headers = request.find('{http://www.example.org/ECMLPKDD}headers')

    if query is not None:
        query_text = query.text
    else:
        query_text = ""
    headers = request.find('{http://www.example.org/ECMLPKDD}headers')
    body = request.find('{http://www.example.org/ECMLPKDD}body')
    if body is not None:
        body_text= body.text
    else:
        body_text = ""

    header_string = headers.text
    
    headers = parser.parsestr(header_string)
    # Extract the desired headers
    host = headers.get("Host")
    connection = headers.get("Connection")
    accept = headers.get("Accept")
    accept_charset = headers.get("Accept-Charset")
    accept_language = headers.get("Accept-Language")
    cache_control = headers.get("Cache-control")
    cookie = headers.get("Cookie")
    pragma = headers.get("Pragma")
    user_agent = headers.get("User-Agent")
    content_length = headers.get("Content-Length")
    content_type = headers.get("Content-Type")
    
    
    label = labels[type_.text]
    docs.append( (label,method.text,uri.text,query_text,protocol.text,host,connection,accept,
                  accept_charset,accept_language,cache_control,cookie,pragma,user_agent,content_length,content_type) )


df  = pd.DataFrame.from_records(docs,columns=['label','method','uri','query','protocol','host','connection','accept',
                  'accept_charset','accept_language','cache_control','cookie','pragma','user_agent','content_length','content_type'])
df['label'].apply(int)

dataset = df.sample(frac=1)
dataset.to_csv('emcl_extract.csv',header=True,index=False,quoting=csv.QUOTE_ALL,columns=['label','method','uri','query','protocol','host','connection','accept',
                  'accept_charset','accept_language','cache_control','cookie','pragma','user_agent','content_length','content_type'])


#print("Training data length: "+str(len(dataset['request'])))



#print("MAX input length: "+str(max_length))

#print number of samples per class  in training data
dataset['label'].value_counts()

0    35006
6     2302
5     2295
4     2279
3     2279
2     2274
7     1856
1     1825
Name: label, dtype: int64